In [0]:
%sql
use catalog dev; 
use schema silver; 

In [0]:
%sql
CREATE OR REPLACE TABLE heartrate_device (device_id INT, mrn STRING, name STRING, time TIMESTAMP, heartrate DOUBLE); 

In [0]:
%sql
INSERT INTO heartrate_device VALUES 
(23, "40580129", "Nicholas Spears", "2020-02-01T00:01:58.000+0000", 54.0122153343), 
(17, "52804177", "Lynn Russell", "2020-02-01T00:02:55.000+0000", 92.5136468131), 
(37, "65300842", "Samuel Hughes", "2020-02-01T00:08:58.000+0000", 52.1354807863), 
(23, "40580129", "Nicholas Spears", "2020-02-01T00:16:51.000+0000", 54.6477014191), 
(17, "52804177", "Lynn Russell", "2020-02-01T00:18:08.000+0000", 95.033344842); 

In [0]:
%sql
SELECT * FROM heartrate_device ;

 

In [0]:
%sql
CREATE OR REPLACE VIEW gold.agg_heartrate AS ( 
SELECT mrn, name, MEAN(heartrate) avg_heartrate, DATE_TRUNC("DD", time) date 
FROM heartrate_device 
GROUP BY mrn, name, DATE_TRUNC("DD", time) 
); 

In [0]:
%sql
SELECT * FROM gold.agg_heartrate ;

In [0]:
grant use catalog on catalog dev to `account users`; 
grant use schema on schema dev.gold to `account users`; 
grant select on table dev.gold.agg_heartrate to `account users` 



In [0]:
%sql

-- Column level 

CREATE OR REPLACE VIEW gold.agg_heartrate AS 
SELECT 
CASE WHEN 
is_account_group_member(‘account users’) THEN mrn 
ELSE 'REDACTED' 
END AS mrn, 
name, 
MEAN(heartrate) avg_heartrate, 
DATE_TRUNC("DD", time) date 
FROM heartrate_device 
GROUP BY mrn, name, DATE_TRUNC("DD", time) ;

 

 



In [0]:
%sql
-- row level 

CREATE OR REPLACE VIEW gold.agg_heartrate AS 

SELECT 

mrn, 

time, 

device_id, 

heartrate 

FROM heartrate_device 

WHERE 

CASE WHEN 

is_account_group_member('account users') THEN device_id < 30 

ELSE TRUE 

END 

 

Data masking function 

CREATE OR REPLACE FUNCTION datamask(x STRING) 

RETURNS STRING 

RETURN CONCAT(REPEAT("*", LENGTH(x) - 2), RIGHT(x, 2) 

); 

 



In [0]:
%sql
CREATE or replace VIEW gold.agg_heartrate AS 

SELECT 

CASE WHEN 

is_account_group_member('account users') THEN datamask(mrn) 

ELSE mrn 

END AS mrn, 

time, 

device_id, 

heartrate 

FROM heartrate_device 